# Instamart Scraper

In [ ]:
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

In [ ]:
a = pd.read_excel('Swiggy_Pincode_OOS.xlsx')

In [ ]:
a

In [ ]:
a = a.sort_values(by=["DRR_GCS"],ascending = False)

In [ ]:
b = a[0:500]

In [ ]:
b = b.sort_values(by=["Pincode"])

In [ ]:
b.reset_index(inplace=True)

In [ ]:

b.drop('index',axis=1,inplace=True)

In [ ]:
b

In [ ]:
path = r"C:\Users\Sanchit\Downloads\chromedriver.exe"
mobile_emulation = {
   "deviceMetrics": { "width": 360, "height": 640, "pixelRatio": 3.0 },
   "userAgent": "Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166 Mobile Safari/535.19" }

In [ ]:
options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
options.add_argument("--disable-javascript")
options.add_argument('--headless')
options.add_argument('--disable-gpu')  
options.add_argument('--no-sandbox')

In [ ]:
count = 0
df = {"Date":[],"Master SKU":[],"Category":[],"City":[],"Pincode":[],"Selling Price":[],"Availability":[],"STORE_ID":[],"Opportunity Lost":[]}

In [ ]:
start_time = time.time()

In [ ]:
i = 419

In [ ]:
while i < 500:
    browser = webdriver.Chrome(executable_path=path, chrome_options = options)
    browser.get(b.loc[i,"Link"])
    element = browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/button')
    element.click()
    sleep(2)
    browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div/div[1]/label/input').send_keys(str(b.loc[i,"Pincode"]))
    sleep(2)
    browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[2]/div/div/button/span').click()
    sleep(5)
    x = browser.page_source
    soup = BeautifulSoup(x,"html.parser")
    soup.beautify
    
    try:
        oos = soup.find("span",{"class":"_2z1M0 _1V2V5"})
        availability = oos.text
        sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
    except AttributeError:
        try:
            oos = soup.find("div",{"class":"_1IsDA"}).text
            if oos == 'go to swiggy home':
                availability = "Out of Stock"
            sp = 0
        except AttributeError:
            try:
                sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                availability = "In Stock"
            except AttributeError:
                try:
                    ta = soup.find("div",{"class":"_2xPHa _2loQy"})
                    if ta.text == "Try Again":
                        sp = (0)
                        availability = "Out of Stock"
                except AttributeError:
                    browser.get(b.loc[i,"Link"])
                    sleep(3)
                    x = browser.page_source
                    soup = BeautifulSoup(x,"html.parser")
                    soup.beautify
                    sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                    availability = "In Stock"
                
    count = count+1
    print(count)            
    df["Selling Price"].append(sp)
    df["Availability"].append(availability)
    df["Date"].append(datetime.now())
    df['Master SKU'].append(b.loc[i,"Master SKU"])
    df["Category"].append(b.loc[i,"Category"])
    df["City"].append(b.loc[i,"City"])
    df["Pincode"].append(b.loc[i,"Pincode"])
    df["STORE_ID"].append(b.loc[i,"STORE_ID"])
    df["Opportunity Lost"].append(b.loc[i,"DRR_GCS"])
    
    j = i+1
    while b.loc[j,"Pincode"] == b.loc[j-1,"Pincode"]:
        browser.get(b.loc[j,"Link"])
        sleep(3)
        x = browser.page_source
        soup = BeautifulSoup(x,"html.parser")
        soup.beautify
        try:
            oos = soup.find("span",{"class":"_2z1M0 _1V2V5"})
            availability = oos.text
            sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text 
        except AttributeError:
            try:
                sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                availability = "In Stock"
            except AttributeError:
                try:
                    sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                    availability = "In Stock"
                except AttributeError:
                    try:
                        ta = soup.find("div",{"class":"_2xPHa _2loQy"})
                        if ta.text == "Try Again":
                            sp = (0)
                            availability = "Out of Stock"
                    except AttributeError:
                        browser.get(b.loc[j,"Link"])
                        sleep(3)
                        x = browser.page_source
                        soup = BeautifulSoup(x,"html.parser")
                        soup.beautify
                        sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                        availability = "In Stock"
        
        df["Selling Price"].append(sp)
        df["Availability"].append(availability)
        df["Date"].append(datetime.now())
        df['Master SKU'].append(b.loc[j,"Master SKU"])
        df["Category"].append(b.loc[j,"Category"])
        df["City"].append(b.loc[j,"City"])
        df["Pincode"].append(b.loc[j,"Pincode"])
        df["STORE_ID"].append(b.loc[j,"STORE_ID"])
        df["Opportunity Lost"].append(b.loc[j,"DRR_GCS"])
        j = j + 1
        count+=1
        print(count)
    
    i = j
    browser.close()
        
    

In [ ]:
fin = pd.DataFrame(df)

In [ ]:
fin